In [1]:
# Import modules
import sys

In [2]:
# Import custom modules
sys.path.append('../')

from src.models.upload_data import *

In [3]:
# Set notebook attributes
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Set magic attributes
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Configuration
args = argparse.ArgumentParser()
args.add_argument("--config", default="../params.yaml")
parsed_args = vars(args.parse_args(""))
config_path = parsed_args['config']

### Till Activity

In [6]:
%%time
# Data ingestion, cleaning/wrangling
df_meta_raw = pull_data(config_path, metabase_sql(), 'DWH', 'metabase', False, 'raw')

Currently loading metabase data set ...


,account_no,amount,balance_after,client_name,comments,created_at,db_transaction_id,phone,status,store_number,transaction_id,transaction_time,type,updated_at
230405,None,195.00,1497.38,KENNETH JAIRO OTWOMBE,None,2022-08-23 11:18:55,18645980,+254728753117,0,727417,QHN0NWGS1A,2022-08-23 14:18:55,c2b,2022-08-23 11:18:57
269100,None,120.00,821.85,LEAH NJERI OKANGA,None,2022-08-23 16:37:30,18691210,+254716717052,0,7591884,QHN8OMMT2O,2022-08-23 19:37:30,c2b,2022-08-23 16:37:35


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 875 ms
Wall time: 1.35 s


In [7]:
%%time
# Data ingestion, cleaning/wrangling
df_meta_clean = clean_dataset(config_path, 'metabase')


Currently cleaning metabase data set ...
Analysis start date 2022-08-16 00:00:00
Analysis latest date 2022-08-24 00:00:00


,store_number,phone,transaction_id,transaction_time
76055,7970995,+254703930166,QHJ0GJKCGW,2022-08-19
111808,760958,+254708766812,QHJ8HD0GVQ,2022-08-19


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 4.38 s
Wall time: 8.17 s


In [8]:
%%time
# Data aggregation
df_meta_aggregate = meta_agg_summaries(config_path)

,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val
73856,7750776,2022-10-26,2022-11-24,28,7771.26
80455,7840152,2022-11-09,2022-11-09,1,50.00


CPU times: total: 328 ms
Wall time: 914 ms


In [9]:
%%time
# Feature engineering
df_meta_features = meta_feature_engineering(config_path, extract_end_date)


Expected transaction days feature sample:
  store_number last_trx_date most_recent_trx_date_past_30_days  \
6       105714    2022-11-24                        2022-10-26   

   expected_trx_days  
6              30.00  
---------------------------------------------------------------------------------------------------------------------------------------

Consistency feature sample:
  store_number  expected_trx_days  actual_trx_days  page_active_days
6       105714                 30               29              0.97
---------------------------------------------------------------------------------------------------------------------------------------

Number of days since last transaction feature sample:
  store_number  days_since_last_trx last_trx_date
6       105714                 0.00    2022-11-24
---------------------------------------------------------------------------------------------------------------------------------------


,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency
105233,7234236,NaT,NaT,0,116.67,0,0.00,No_rules_relaxed,NaN,0,0.00
33865,7237972,2022-10-26,2022-11-24,26,10147.55,30,0.87,No_rules_relaxed,0.00,Yes,1.00


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 797 ms
Wall time: 2.14 s


In [10]:
# Final dataset schema
df_meta_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115150 entries, 0 to 115149
Data columns (total 11 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   store_number                       115150 non-null  object        
 1   most_recent_trx_date_past_30_days  99685 non-null   datetime64[ns]
 2   last_trx_date                      99685 non-null   datetime64[ns]
 3   actual_trx_days                    115150 non-null  int32         
 4   approx_30_days_trx_val             115150 non-null  float64       
 5   expected_trx_days                  115150 non-null  int32         
 6   page_active_days                   115150 non-null  float64       
 7   inference_col                      115150 non-null  object        
 8   days_since_last_trx                99685 non-null   float64       
 9   transacted_last_5_days             115150 non-null  object        
 10  weight_till_recency 